In [1]:
%%capture
!apt-get install maven -qq

In [2]:
%%capture
!git clone --recurse-submodules https://github.com/castorini/anserini.git
%cd anserini
!cd tools/eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../../..
!mvn clean package appassembler:assemble -DskipTests -Dmaven.javadoc.skip=true

In [3]:
!ls target

anserini-0.11.1-SNAPSHOT-fatjar.jar  classes		     maven-archiver
anserini-0.11.1-SNAPSHOT.jar	     generated-sources	     maven-status
appassembler			     generated-test-sources  test-classes


In [4]:
%%capture
!wget https://git.uwaterloo.ca/jimmylin/anserini-indexes/raw/master/index-msmarco-passage-20201117-f87c94.tar.gz


In [5]:
%%capture
!tar xvfz index-msmarco-passage-20201117-f87c94.tar.gz -C indexes/

In [6]:
%%script bash
cd indexes/index-msmarco-passage-20201117-f87c94/
ls

_9.fdt
_9.fdx
_9.fnm
_9_Lucene50_0.doc
_9_Lucene50_0.tim
_9_Lucene50_0.tip
_9_Lucene80_0.dvd
_9_Lucene80_0.dvm
_9.nvd
_9.nvm
_9.si
segments_2
write.lock


In [7]:
!du -h indexes/index-msmarco-passage-20201117-f87c94

2.6G	indexes/index-msmarco-passage-20201117-f87c94


In [8]:
%%script bash
cd tools/topics-and-qrels/
ls

docids.covid.round1.txt
docids.covid.round2.txt
docids.covid.round3.txt
docids.covid.round4.txt
docids.covid.round5.txt
prels.mq.10001-20000.txt
prels.mq.1-10000.txt
prels.mq.20001-60000.txt
prels.web.1-50.txt
qrels.adhoc.101-150.txt
qrels.adhoc.151-200.txt
qrels.adhoc.401-450.txt
qrels.adhoc.451-550.txt
qrels.adhoc.51-100.txt
qrels.backgroundlinking18.txt
qrels.backgroundlinking19.txt
qrels.cacm.txt
qrels.car17v1.5.benchmarkY1test.txt
qrels.car17v2.0.benchmarkY1test.txt
qrels.clef06fr.txt
qrels.core17.txt
qrels.core18.txt
qrels.covid-complete.txt
qrels.covid-round12.txt
qrels.covid-round1.txt
qrels.covid-round2-cumulative.txt
qrels.covid-round2.txt
qrels.covid-round3-cumulative.txt
qrels.covid-round3.txt
qrels.covid-round4-cumulative.txt
qrels.covid-round4.txt
qrels.covid-round5.txt
qrels.dl19-doc.txt
qrels.dl19-passage.txt
qrels.dl20-doc.txt
qrels.dl20-passage.txt
qrels.fire12bn.176-225.txt
qrels.fire12en.176-225.txt
qrels.fire12hi.176-225.txt
qrels.microblog2011.txt
qrels.microblog2

In [9]:
pip install pyserini==0.11.0.0

     |████████████████████████████████| 67.1MB 56kB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 1.1MB 38.3MB/s 
     |████████████████████████████████| 2.1MB 36.5MB/s 
     |████████████████████████████████| 7.9MB 22.9MB/s 
     |████████████████████████████████| 901kB 39.4MB/s 
     |████████████████████████████████| 3.3MB 45.0MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [10]:
%%script bash
python -m pyserini.search --topics msmarco-passage-dev-subset \
 --index indexes/index-msmarco-passage-20201117-f87c94 \
 --output runs/run.msmarco-passage-20201117-f87c94.bm25tuned.txt \
 --bm25 --hits 100 --k1 0.9 --b 0.4

Setting BM25 parameters: k1=0.9, b=0.4
Running msmarco-passage-dev-subset topics, saving to runs/run.msmarco-passage-20201117-f87c94.bm25tuned.txt...


100%|██████████| 6980/6980 [04:04<00:00, 28.56it/s]


In [11]:
%%script bash
cd runs/
ls

run.msmarco-passage-20201117-f87c94.bm25tuned.txt


In [12]:
!tools/eval/trec_eval.9.0.4/trec_eval -c -mrecall.100 -mmap -mndcg tools/topics-and-qrels/qrels.msmarco-passage.dev-subset.txt runs/run.msmarco-passage-20201117-f87c94.bm25tuned.txt

map                   	all	0.1917
recall_100            	all	0.6578
ndcg                  	all	0.2870
